In [4]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [5]:
# Set the file path
filepath = Path("../Resources/amzn_data.csv")

# Read the CSV located at the file path into a Pandas DataFrame
# Set the `Date` column as the index and auto-format the datetime string
aapl_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

# Print the DataFrame
aapl_df.head()

,Date,Open,High,Low,Close,Volume,Volatility,sentiment,RSI
0,2019-01-02,1465.20,1553.36,1460.9300,1536.730,7132821,0.902244,0.005004,NaN
1,2019-01-03,1520.01,1538.00,1498.1062,1502.070,6340704,0.200930,-0.030889,NaN
2,2019-01-04,1530.00,1594.00,1518.3100,1574.540,8285596,0.652522,0.079218,NaN
3,2019-01-07,1602.31,1634.56,1589.1850,1631.120,7252880,0.369340,0.086756,NaN
4,2019-01-08,1664.69,1676.61,1616.6100,1655.835,8184304,0.127908,0.051332,NaN


In [8]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = aapl_df.loc[:, ["Date", "Close"]].copy()

# Set the short window and long windows
short_window = 5
long_window = 15

# Set the `date` column as the index
signals_df = signals_df.set_index("Date", drop=True)

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df["SMA50"] = signals_df["Close"].rolling(window=short_window).mean()
signals_df["SMA100"] = signals_df["Close"].rolling(window=long_window).mean()
signals_df["Signal"] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df["Signal"][short_window:] = np.where(
    signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df["Entry/Exit"] = signals_df["Signal"].diff()

signals_df.head(10)

,Close,SMA50,SMA100,Signal,Entry/Exit
Date,,,,,
2019-01-02,1536.7300,NaN,NaN,0.0,NaN
2019-01-03,1502.0700,NaN,NaN,0.0,0.0
2019-01-04,1574.5400,NaN,NaN,0.0,0.0
2019-01-07,1631.1200,NaN,NaN,0.0,0.0
2019-01-08,1655.8350,1580.05900,NaN,0.0,0.0
2019-01-09,1657.5900,1604.23100,NaN,0.0,0.0
2019-01-10,1655.5000,1634.91700,NaN,0.0,0.0
2019-01-11,1639.6600,1647.94100,NaN,0.0,0.0
2019-01-14,1619.8767,1645.69234,NaN,0.0,0.0


In [9]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Close'].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Close'].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['Close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [Date]   (Close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [Date]   (value)
   .Scatter.Close.I  :Scatter   [Date]   (Close)
   .Scatter.Close.II :Scatter   [Date]   (Close)